# Analysis of CSECICIDS2018

In [1]:
# Data Loading
import pandas as pd
import os
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq

In [2]:
filenames = [f for f in os.listdir('data/CSECICIDS2018_improved') if f.endswith('.csv')]

In [ ]:
active_filenames = [
    'Thursday-22-02-2018.csv',
    'Friday-16-02-2018.csv',
    # 'Thursday-15-02-2018.csv',
    'Wednesday-14-02-2018.csv',
    'Tuesday-20-02-2018.csv',
    'Wednesday-21-02-2018.csv',
    # 'Friday-23-02-2018.csv',
    # 'Wednesday-28-02-2018.csv',
    # 'Friday-02-03-2018.csv',
    # 'Thursday-01-03-2018.csv'
]

In [8]:
EDGE_COLS = [
    'Bwd Packet Length Min', 'Protocol', 'Bwd Packets/s', 'FWD Init Win Bytes',
    'Packet Length Std', 'FIN Flag Count',
    'Packet Length Min', 'Fwd Seg Size Min',
    'Bwd IAT Total', 'SYN Flag Count', 'Bwd Packet Length Std'
]
LABEL_COL = "Label"
ID_COLS = ['Src IP', 'Dst IP', 'Timestamp']
COLS_TO_KEEP = EDGE_COLS + [LABEL_COL] + ID_COLS

In [5]:
data_dir = 'data/CSECICIDS2018_improved'
output_dir = 'data/processed_chunks'
final_path = 'data/combined_active.parquet'
os.makedirs(output_dir, exist_ok=True)

In [9]:
# We'll stream CSV files in chunks and record the earliest Timestamp when an IP (Src or Dst) appears with a non-BENIGN label.
# This avoids loading large files into memory and produces small summary tables we can use to build a split where IPs don't overlap.
chunksize = 200_000
# dictionaries to hold earliest non-benign timestamp per IP
earliest_nonbenign_src = {}  # ip -> pd.Timestamp
earliest_nonbenign_dst = {}  # ip -> pd.Timestamp

for i, filename in enumerate(active_filenames, 1):
    file_path = os.path.join(data_dir, filename)
    print(f'[{i}/{len(active_filenames)}] Scanning: {filename}')

    for chunk in pd.read_csv(file_path, usecols=COLS_TO_KEEP, chunksize=chunksize):
        # parse Timestamp column to datetime (in-place) -- faster to convert only the rows we need
        chunk['Timestamp'] = pd.to_datetime(chunk['Timestamp'], errors='coerce')
        # filter non-BENIGN rows only (we only care about malicious labels)
        nonbenign = chunk[chunk[LABEL_COL] != 'BENIGN']
        if nonbenign.empty:
            continue

        # for Src IPs: for each IP, keep the minimum timestamp seen so far
        src_group = nonbenign.groupby('Src IP')['Timestamp'].min()
        for ip, ts in src_group.items():
            if pd.isna(ts):
                continue
            prev = earliest_nonbenign_src.get(ip)
            if (prev is None) or (ts < prev):
                earliest_nonbenign_src[ip] = ts

        # for Dst IPs
        dst_group = nonbenign.groupby('Dst IP')['Timestamp'].min()
        for ip, ts in dst_group.items():
            if pd.isna(ts):
                continue
            prev = earliest_nonbenign_dst.get(ip)
            if (prev is None) or (ts < prev):
                earliest_nonbenign_dst[ip] = ts

# convert dictionaries to DataFrames
src_df = (pd.DataFrame.from_dict(earliest_nonbenign_src, orient='index', columns=['first_nonbenign_ts'])
          .reset_index().rename(columns={'index': 'ip'}))
dst_df = (pd.DataFrame.from_dict(earliest_nonbenign_dst, orient='index', columns=['first_nonbenign_ts'])
          .reset_index().rename(columns={'index': 'ip'}))

# make sure Timestamp column is datetime dtype
src_df['first_nonbenign_ts'] = pd.to_datetime(src_df['first_nonbenign_ts'])
dst_df['first_nonbenign_ts'] = pd.to_datetime(dst_df['first_nonbenign_ts'])

# write summaries to parquet for later use
os.makedirs(output_dir, exist_ok=True)
src_summary = os.path.join(output_dir, 'earliest_nonbenign_src.parquet')
dst_summary = os.path.join(output_dir, 'earliest_nonbenign_dst.parquet')
src_df.to_parquet(src_summary, index=False)
dst_df.to_parquet(dst_summary, index=False)

print(f'Saved Src summary: {src_summary} ({len(src_df)} rows)')
print(f'Saved Dst summary: {dst_summary} ({len(dst_df)} rows)')

# Optionally, create a combined IP-level table that records first appearance either as src or dst
combined = pd.concat([src_df.assign(role='src'), dst_df.assign(role='dst')], ignore_index=True)
combined = combined.sort_values(['ip', 'first_nonbenign_ts']).drop_duplicates('ip', keep='first')
combined_path = os.path.join(output_dir, 'earliest_nonbenign_ip.parquet')
combined.to_parquet(combined_path, index=False)
print(f'Saved combined IP summary: {combined_path} ({len(combined)} rows)')

[1/10] Scanning: Thursday-22-02-2018.csv
[2/10] Scanning: Friday-16-02-2018.csv
[2/10] Scanning: Friday-16-02-2018.csv
[3/10] Scanning: Thursday-15-02-2018.csv
[3/10] Scanning: Thursday-15-02-2018.csv
[4/10] Scanning: Wednesday-14-02-2018.csv
[4/10] Scanning: Wednesday-14-02-2018.csv
[5/10] Scanning: Tuesday-20-02-2018.csv
[5/10] Scanning: Tuesday-20-02-2018.csv
[6/10] Scanning: Wednesday-21-02-2018.csv
[6/10] Scanning: Wednesday-21-02-2018.csv
[7/10] Scanning: Friday-23-02-2018.csv
[7/10] Scanning: Friday-23-02-2018.csv
[8/10] Scanning: Wednesday-28-02-2018.csv
[8/10] Scanning: Wednesday-28-02-2018.csv
[9/10] Scanning: Friday-02-03-2018.csv
[9/10] Scanning: Friday-02-03-2018.csv
[10/10] Scanning: Thursday-01-03-2018.csv
[10/10] Scanning: Thursday-01-03-2018.csv
Saved Src summary: data/processed_chunks/earliest_nonbenign_src.parquet (30 rows)
Saved Dst summary: data/processed_chunks/earliest_nonbenign_dst.parquet (49 rows)
Saved combined IP summary: data/processed_chunks/earliest_nonbe

In [12]:
combined.sort_values('first_nonbenign_ts').reset_index(drop=True)

,ip,first_nonbenign_ts,role
0,172.31.69.25,2018-02-14 14:33:26.718731,dst
1,18.221.219.4,2018-02-14 14:33:26.718731,src
2,13.58.98.64,2018-02-14 18:01:21.199541,src
3,18.219.211.138,2018-02-15 13:27:42.466479,src
4,18.217.165.70,2018-02-15 15:00:12.551313,src
5,13.59.126.31,2018-02-16 14:12:14.134225,src
6,18.219.193.20,2018-02-16 17:45:27.826666,src
7,52.14.136.135,2018-02-20 14:13:54.662633,src
8,18.218.55.126,2018-02-20 14:13:57.703554,src
9,18.218.229.235,2018-02-20 14:13:59.956625,src


In [ ]:
dataset = pq.ParquetDataset(final_path)
table = dataset.read()

In [ ]:
table.nbytes / (10 ** 9)  # Size in GB

5.240525175

In [ ]:
COLS_TO_LEAVE = ID_COLS + [LABEL_COL]
df = table.select(COLS_TO_LEAVE).to_pandas()

In [ ]:
df.head()

,Src IP,Dst IP,Timestamp,Label
0,172.31.66.11,23.15.8.121,2018-02-22 12:22:51.109054,BENIGN
1,172.31.66.11,23.52.91.27,2018-02-22 12:22:51.575427,BENIGN
2,172.31.66.11,72.21.91.29,2018-02-22 12:22:53.312634,BENIGN
3,172.31.66.11,108.174.11.1,2018-02-22 12:22:57.954629,BENIGN
4,172.31.66.11,52.179.17.38,2018-02-22 12:23:33.459731,BENIGN


In [ ]:
df.shape

(32377064, 4)

In [ ]:
df['target'] = np.where(df[LABEL_COL] == 'BENIGN', 0, 1)

In [ ]:
df['target'].value_counts(normalize=True)

target
0    0.889708
1    0.110292
Name: proportion, dtype: float64

In [ ]:
df['Timestamp'] = pd.to_datetime(df['Timestamp'])

In [ ]:
df['day'] = df['Timestamp'].dt.day

In [ ]:
active_filenames

['Thursday-22-02-2018.csv',
 'Friday-16-02-2018.csv',
 'Wednesday-14-02-2018.csv',
 'Tuesday-20-02-2018.csv',
 'Wednesday-21-02-2018.csv']

In [ ]:
df['day'].unique()

array([22, 23, 16, 14, 15, 20, 21], dtype=int32)

In [ ]:
df.sort_values('Timestamp', inplace=True)
df.reset_index(drop=True, inplace=True)

In [ ]:
df['Label'].value_counts(normalize=True)

Label
BENIGN                                  8.897082e-01
DoS Hulk                                5.569251e-02
DDoS-HOIC                               3.342777e-02
FTP-BruteForce - Attempted              9.231041e-03
DDoS-LOIC-HTTP                          8.936202e-03
SSH-BruteForce                          2.909374e-03
DDoS-LOIC-UDP                           7.804908e-05
DDoS-LOIC-UDP - Attempted               7.752402e-06
DoS Hulk - Attempted                    2.656201e-06
Web Attack - Brute Force - Attempted    2.347341e-06
Web Attack - Brute Force                2.131138e-06
Web Attack - XSS                        1.235442e-06
Web Attack - SQL                        4.941770e-07
Web Attack - SQL - Attempted            1.235442e-07
Web Attack - XSS - Attempted            9.265819e-08
Name: proportion, dtype: float64

In [ ]:
df.groupby('Label')['Src IP'].nunique()

Label
BENIGN                                  80677
DDoS-HOIC                                  10
DDoS-LOIC-HTTP                             10
DDoS-LOIC-UDP                              10
DDoS-LOIC-UDP - Attempted                   2
DoS Hulk                                    1
DoS Hulk - Attempted                        1
FTP-BruteForce - Attempted                  3
SSH-BruteForce                              1
Web Attack - Brute Force                    1
Web Attack - Brute Force - Attempted        1
Web Attack - SQL                            1
Web Attack - SQL - Attempted                1
Web Attack - XSS                            1
Web Attack - XSS - Attempted                1
Name: Src IP, dtype: int64

In [ ]:
df.head()

,Src IP,Dst IP,Timestamp,Label,target,day
0,172.31.66.58,239.255.255.250,2018-02-14 12:28:07.743746,BENIGN,0,14
1,172.31.66.46,239.255.255.250,2018-02-14 12:28:08.143839,BENIGN,0,14
2,172.31.66.46,169.254.169.254,2018-02-14 12:28:08.175858,BENIGN,0,14
3,107.217.94.48,172.31.66.46,2018-02-14 12:28:08.214861,BENIGN,0,14
4,172.31.66.46,172.31.0.2,2018-02-14 12:28:08.295304,BENIGN,0,14


In [ ]:
train_ids = df.index.tolist()[:int(0.8 * len(df))]
test_ids =  df.index.tolist()[int(0.8 * len(df)):]
train_df = df.loc[train_ids]
test_df = df.loc[test_ids]

In [ ]:
malicious_ratio_train = train_df['target'].value_counts(normalize=True)[1]
malicious_ratio_test = test_df['target'].value_counts(normalize=True)[1]

print(f"Malicious ratio in train set: {malicious_ratio_train:.6f}")
print(f"Malicious ratio in test set: {malicious_ratio_test:.6f}")

Malicious ratio in train set: 0.137857
Malicious ratio in test set: 0.000032


In [ ]:
malicious_count_train = train_df['target'].value_counts()[1]
malicious_count_test = test_df['target'].value_counts()[1]

int(malicious_count_train), int(malicious_count_test)

(3570716, 208)

In [ ]:
print(f"Days in the training set: {train_df['day'].unique()}")
print(f"Days in the test set: {test_df['day'].unique()}")

Days in the training set: [14 15 16 20 21]
Days in the test set: [21 22 23]


In [ ]:
print(f"Day 21 is both in training and test sets. {train_df[train_df['day'].isin([21])].shape[0]/train_df.shape[0]:.4f}, {test_df[test_df['day'].isin([21])].shape[0]/test_df.shape[0]:.4f}")

Day 21 is both in training and test sets. 0.2532, 0.0624


In [ ]:
df[df['target'] == 1]

,Src IP,Dst IP,Timestamp,Label,target,day
1388290,18.221.219.4,172.31.69.25,2018-02-14 14:33:26.718731,FTP-BruteForce - Attempted,1,14
1388291,18.221.219.4,172.31.69.25,2018-02-14 14:33:26.719116,FTP-BruteForce - Attempted,1,14
1388292,18.221.219.4,172.31.69.25,2018-02-14 14:33:26.719124,FTP-BruteForce - Attempted,1,14
1388293,18.221.219.4,172.31.69.25,2018-02-14 14:33:26.719307,FTP-BruteForce - Attempted,1,14
1388294,18.221.219.4,172.31.69.25,2018-02-14 14:33:26.719366,FTP-BruteForce - Attempted,1,14
...,...,...,...,...,...,...
31701422,18.218.115.60,172.31.69.28,2018-02-22 20:20:54.257210,Web Attack - SQL,1,22
31712126,18.218.115.60,172.31.69.28,2018-02-22 20:22:00.937247,Web Attack - SQL,1,22
31713293,18.218.115.60,172.31.69.28,2018-02-22 20:22:07.992257,Web Attack - SQL,1,22
31745127,18.218.115.60,172.31.69.28,2018-02-22 20:24:42.266874,Web Attack - SQL,1,22


In [ ]:
malicious_senders_train = set(train_df[train_df['target'] == 1]['Src IP'].unique())
malicious_senders_test = set(test_df[test_df['target'] == 1]['Src IP'].unique())

malicious_nodes_train = malicious_senders_train.union(set(train_df[train_df['target'] == 1]['Dst IP'].unique()))
malicious_nodes_test = malicious_senders_test.union(set(test_df[test_df['target'] == 1]['Dst IP'].unique()))

common_malicious_senders = malicious_senders_train.intersection(malicious_senders_test)
common_malicious_nodes = malicious_nodes_train.intersection(malicious_nodes_test)

print("IP-level statistics")
print(f"Total malicious senders in train set: {len(malicious_senders_train)}")
print(f"Total malicious senders in test set: {len(malicious_senders_test)}")
print(f"Common malicious senders: {len(common_malicious_senders)}")

print(f"Total malicious nodes in train set: {len(malicious_nodes_train)}")
print(f"Total malicious nodes in test set: {len(malicious_nodes_test)}")
print(f"Common malicious nodes: {len(common_malicious_nodes)}")

IP-level statistics
Total malicious senders in train set: 16
Total malicious senders in test set: 1
Common malicious senders: 1
Total malicious nodes in train set: 16
Total malicious nodes in test set: 2
Common malicious nodes: 2


In [ ]:
# computing the earliest index when each malicious IP appears in the whole dataset
malicious_ips = malicious_nodes_train.union(malicious_nodes_test)
malicious_senders = malicious_senders_train.union(malicious_senders_test)

malicious_idx_dict = {}
malicious_sender_idx_dict = {}
for ip in malicious_ips:
    if ip in malicious_senders:
        ip_df = df[(df['Src IP'] == ip)]
        first_index = ip_df.index.min()
        malicious_sender_idx_dict[ip] = first_index
    ip_df = df[(df['Src IP'] == ip) | (df['Dst IP'] == ip)]
    first_index = ip_df.index.min()
    malicious_idx_dict[ip] = first_index

malicious_idx_series = pd.Series(malicious_idx_dict).sort_values()
malicious_sender_idx_series = pd.Series(malicious_sender_idx_dict).sort_values()


In [ ]:
(malicious_idx_series / len(df) * 100).rename("First appearance index (%)").round(2)

172.31.69.28       0.05
172.31.69.25       0.10
18.221.219.4       4.29
13.58.98.64       11.47
13.59.126.31      21.39
18.219.193.20     28.24
52.14.136.135     44.71
18.218.55.126     44.71
18.218.229.235    44.72
18.219.9.1        44.72
18.216.200.189    44.72
18.218.115.60     44.72
18.219.5.43       44.73
18.216.24.42      44.73
18.218.11.51      44.73
18.219.32.43      44.74
Name: First appearance index (%), dtype: float64

In [ ]:
malicious_sender_idx_series / len(df) * 100

172.31.69.28       0.050616
172.31.69.25       0.102119
18.221.219.4       4.287881
13.58.98.64       11.466682
13.59.126.31      21.393530
18.219.193.20     28.238700
52.14.136.135     44.713418
18.218.55.126     44.714848
18.218.229.235    44.716102
18.219.9.1        44.717152
18.216.200.189    44.719574
18.218.115.60     44.721717
18.219.5.43       44.727752
18.216.24.42      44.729204
18.218.11.51      44.732231
18.219.32.43      44.738677
dtype: float64